In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAI
from langchain.chains import create_sql_query_chain

load_dotenv()

True

# [Build a Question/Answering system over SQL data](https://python.langchain.com/v0.2/docs/tutorials/sql_qa/)


## Architecture

At a high-level, the steps of these systems are:  

Convert question to DSL query: Model converts user input to a SQL query.  
Execute SQL query: Execute the query.  
Answer the question: Model responds to user input using the query results.  

<img src='./assets/imgs/sql.bmp' ></img>



In [3]:
# create sqlite
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('sqlite:///iris.db', echo=False)

df = pd.read_csv('./data/iris.csv')
df.to_sql(name='iris', con=engine, if_exists='replace')

150

In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///iris.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM iris LIMIT 10;")

sqlite
['iris']


"[(0, 5.1, 3.5, 1.4, 0.2, 'Setosa'), (1, 4.9, 3.0, 1.4, 0.2, 'Setosa'), (2, 4.7, 3.2, 1.3, 0.2, 'Setosa'), (3, 4.6, 3.1, 1.5, 0.2, 'Setosa'), (4, 5.0, 3.6, 1.4, 0.2, 'Setosa'), (5, 5.4, 3.9, 1.7, 0.4, 'Setosa'), (6, 4.6, 3.4, 1.4, 0.3, 'Setosa'), (7, 5.0, 3.4, 1.5, 0.2, 'Setosa'), (8, 4.4, 2.9, 1.4, 0.2, 'Setosa'), (9, 4.9, 3.1, 1.5, 0.1, 'Setosa')]"

### Convert question to SQL query

In [5]:
llm = ChatOpenAI()

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "Qual a média das petalas por tipo de flor"})
response

'SELECT variety, AVG("petal.length") AS avg_petal_length, AVG("petal.width") AS avg_petal_width\nFROM iris\nGROUP BY variety\nORDER BY avg_petal_length DESC;'

In [6]:
# Rodar com o gpt-40-mini 
"""O langchain não está adapdado a esse modelo
"""
# from langchain.schema.runnable import RunnableLambda
# llm = ChatOpenAI(model='gpt-4o-mini', temperature=0) # o gpt-4o-mini Adiciona um subtipo


# write_query = create_sql_query_chain(llm, db)
# chain = write_query | RunnableLambda(lambda x: x[10:]) # Necessário adptar a saída
# response = chain.invoke({"question": "Qual a média de comprimento e largura das petalas por tipo de flor, remover limite"})
# response

'O langchain não está adapdado a esse modelo\n'

We can execute the query to make sure it's valid:

In [7]:
db.run(response)

"[('Virginica', 5.552, 2.026), ('Versicolor', 4.26, 1.3259999999999998), ('Setosa', 1.4620000000000002, 0.2459999999999999)]"

This technique is inspired by papers like this, which suggest showing examples rows and being explicit about tables improves performance. We can also inspect the full prompt like so:

In [8]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### Execute SQL query
Now that we've generated a SQL query, we'll want to execute it. This is the most dangerous part of creating a SQL chain. Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution (see below).

In [9]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "Qual a média das petalas por tipo de flor"})

"[('Virginica', 5.552, 2.026), ('Versicolor', 4.26, 1.3259999999999998), ('Setosa', 1.4620000000000002, 0.2459999999999999)]"

## Answer the question

Now that we've got a way to automatically generate and execute queries, we just need to combine the original question and SQL query result to generate a final answer. We can do this by passing question and result to the LLM once more:

In [10]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke({"question": "Qual a média das petalas por tipo de flor"}))

A média das pétalas por tipo de flor é a seguinte:

- Para a variedade 'Setosa', a média do comprimento da pétala é de aproximadamente 1.462 e a média da largura da pétala é de aproximadamente 0.246.
- Para a variedade 'Versicolor', a média do comprimento da pétala é de 4.26 e a média da largura da pétala é de aproximadamente 1.326.
- Para a variedade 'Virginica', a média do comprimento da pétala é de 5.552 e a média da largura da pétala é de aproximadamente 2.026.


## Agentes

In [11]:
from langchain_community.agent_toolkits import create_sql_agent

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0) 
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
agent_executor.invoke({"input": "Qual a média das petalas por tipo de flor, usar todos os registros"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


iris
Invoking: `sql_db_schema` with `{'table_names': 'iris'}`



CREATE TABLE iris (
	"index" BIGINT, 
	"sepal.length" FLOAT, 
	"sepal.width" FLOAT, 
	"petal.length" FLOAT, 
	"petal.width" FLOAT, 
	variety TEXT
)

/*
3 rows from iris table:
index	sepal.length	sepal.width	petal.length	petal.width	variety
0	5.1	3.5	1.4	0.2	Setosa
1	4.9	3.0	1.4	0.2	Setosa
2	4.7	3.2	1.3	0.2	Setosa
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT variety, AVG(petal.length) as average_petal_length FROM iris GROUP BY variety'}`


```sql
SELECT variety, AVG(petal_length) as average_petal_length FROM iris GROUP BY variety
```
Invoking: `sql_db_query` with `{'query': 'SELECT variety, AVG(petal.length) as average_petal_length FROM iris GROUP BY variety'}`


Error: (sqlite3.OperationalError) no such column: petal.length
[SQL: SELECT variety, AVG(petal.length) as average_petal_length FROM iris GROUP BY variety]
(Back

{'input': 'Qual a média das petalas por tipo de flor, usar todos os registros',
 'output': 'A média do comprimento das pétalas por tipo de flor é a seguinte:\n\n- Setosa: 1.46\n- Versicolor: 4.26\n- Virginica: 5.55'}

In [12]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f78bcd75bb0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f78bcd75bb0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f78bcd75bb0>),
 QuerySQLCheckerTool(description='Use this tool to double check

We will also want to create a system prompt for our agent. This will consist of instructions for how to behave.

In [14]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [23]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

In [24]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Qual a média das petalas por tipo de flor, usar todos os registros")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JFJ48eFb06aygmMpjjAVsSCv', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 556, 'total_tokens': 568}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_611b667b19', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e7fd90e9-c254-4dd9-bdc1-07a33feb2478-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_JFJ48eFb06aygmMpjjAVsSCv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 556, 'output_tokens': 12, 'total_tokens': 568})]}}
----
{'tools': {'messages': [ToolMessage(content='iris', name='sql_db_list_tables', tool_call_id='call_JFJ48eFb06aygmMpjjAVsSCv')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_I4xclNEAAoTI31GWrxwTGTtE', 'function': {'arguments': '{"table_names":"iris"}', 'n